##Get text for comparison

In [ ]:
X ="I love horror movies"
Y ="Lights out is a horror movie"

##Cosine similarity

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def consine_sim(X,Y):

  # tokenization
  X_list = word_tokenize(X)
  Y_list = word_tokenize(Y)

  # sw contains the list of stopwords
  sw = stopwords.words('english')
  l1 =[];l2 =[]

  # remove stop words from the string
  X_set = {w for w in X_list if not w in sw}
  Y_set = {w for w in Y_list if not w in sw}

  # form a set containing keywords of both strings
  rvector = X_set.union(Y_set)
  for w in rvector:
      if w in X_set: l1.append(1) # create a vector
      else: l1.append(0)
      if w in Y_set: l2.append(1)
      else: l2.append(0)
  c = 0

  # cosine formula
  for i in range(len(rvector)):
          c+= l1[i]*l2[i]
  cosine = c / float((sum(l1)*sum(l2))**0.5)

  return cosine

In [ ]:
consine_sim(X,Y)

0.2886751345948129

##NLP metrics

In [ ]:
!pip install aac-metrics
!aac-metrics-download

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 30.8 MB/s eta 0:00:00
  Created wheel for aac-metrics: filename=aac_metrics-0.5.4-py3-none-any.whl size=81166 sha256=b0fa8e713a67feb1016fa10544c699f9f1d6e8a43492673a3cc56cfaeb3fa659
  Stored in directory: /root/.cache/pip/wheels/ec/3d/e2/1d7cc3036cd832bf4578448cb62aa3fcd2132a61edc2b95f8f
Successfully built aac-metrics
2024-10-17 16:49:56.161622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 16:49:56.213760: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuD

In [ ]:
from aac_metrics import evaluate

In [ ]:
def nlp_score(X,Y, metric_name):

  actual=[]
  predicted=[]

  for i in range(5):
    actual.append(X)
    predicted.append(Y)

  # calculate evaluation score
  ##input: list(), list(list(str))
  ##output: dict containing the score of each metric: "bleu_1", "bleu_2", "bleu_3", "bleu_4", "rouge_l", "meteor", "cider_d", "spice", "spider"

  corpus_scores, _ = evaluate([item for item in predicted], [[item] for item in actual])

  return corpus_scores[metric_name]

In [ ]:
nlp_score(X,Y, "spice")

tensor(0.5000, dtype=torch.float64)

In [ ]:
# calculate evaluation score
##input: list(), list(list(str))
##output: dict containing the score of each metric: "bleu_1", "bleu_2", "bleu_3", "bleu_4", "rouge_l", "meteor", "cider_d", "spice", "spider"

# corpus_scores, _ = evaluate([item for item in predicted], [[item] for item in actual])
# print(corpus_scores)

{'bleu_1': tensor(0.1667, dtype=torch.float64), 'bleu_2': tensor(1.8257e-09, dtype=torch.float64), 'bleu_3': tensor(4.3679e-12, dtype=torch.float64), 'bleu_4': tensor(2.2957e-13, dtype=torch.float64), 'meteor': tensor(0.2226, dtype=torch.float64), 'rouge_l': tensor(0.2075, dtype=torch.float64), 'cider_d': tensor(0., dtype=torch.float64), 'spice': tensor(0.5000, dtype=torch.float64), 'spider': tensor(0.2500, dtype=torch.float64)}


## Cosine Similarity (Embedding)

In [ ]:
import random
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Set a random seed
random_seed = 42
random.seed(random_seed)

# Set a random seed for PyTorch (for GPU as well)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

# Load BERT tokenizer and model
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
def bert_embedding(example_sentence, model=model_bert, tokenizer=tokenizer_bert):

  # Tokenize and encode the example sentence
  example_encoding = tokenizer.batch_encode_plus(
      [example_sentence],
      padding=True,
      truncation=True,
      return_tensors='pt',
      add_special_tokens=True
  )
  example_input_ids = example_encoding['input_ids']
  example_attention_mask = example_encoding['attention_mask']

  # Generate embeddings for the example sentence
  with torch.no_grad():
      example_outputs = model(example_input_ids, attention_mask=example_attention_mask)
      example_sentence_embedding = example_outputs.last_hidden_state.mean(dim=1)

  return example_sentence_embedding

In [ ]:
def consine_sim(a,b):
  similarity_score = cosine_similarity(bert_embedding(a), bert_embedding(b))[0][0]
  return similarity_score

In [ ]:
cosine_sim(X,Y)